## 4. Time Series Classification & Prediction Deep Learning
You are provided with daily historical sales data. The task is to forecast the total amount of products sold in every shop for the test set. Note that the list of shops and products slightly changes every month. Creating a robust model that can handle such situations is the goal.
File descriptions
* sales_train.csv - the training set. Daily historical data from January 2013 to October 2015.
* test.csv - the test set. You need to forecast the sales for these shops and products for November 2015.
* sample_submission.csv - a sample submission file in the correct format.
* items.csv - supplemental information about the items/products.
* item_categories.csv - supplemental information about the items categories.
* shops.csv- supplemental information about the shops.

#### Hint:
* Recommended a hybrid CNN-LSTM model 
* Dimensionality Reduction
    - Use Several Classifiers/ Ensemble Method
    - Logistic Regression (with different c values)
    - Random Forest (with different estimator values) 
    - SVM (with different kernels)
    - KNN (with k = 1,2,5,10,20)
    - K (3,5,10) Fold Cross Validation
* Performance Comparison
    - Classification Accuracy, Precision, Recall, Sensitivity, Specificity * AUC, ROC Curve
    - Confusion Matrix

In [21]:
import gc
gc.collect()

107

In [2]:
# Supress warnings
def warn(*args, **kwargs):
    pass
import warnings
import os as os
import numpy as np
import pandas as pd
import re as regx
import gc as gc
import sys as sys
import time as time
import matplotlib
import matplotlib.pyplot as plt

# iPy magic and other cool settings 
%matplotlib inline
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
np.random.seed(42)
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
warnings.warn = warn

from collections import Counter
from scipy.sparse import csr_matrix
from itertools import compress
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, confusion_matrix, classification_report, roc_curve, roc_auc_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [7]:
# Import datasets
item_cats = pd.read_csv('../data/4/item_categories.csv')
items = pd.read_csv('../data/4/items.csv')
sales_train = pd.read_csv('../data/4/sales_train.csv')
test = pd.read_csv('../data/4/test.csv')
shops = pd.read_csv('../data/4/shops.csv')

In [8]:
sales_train.head(5)

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [12]:
# combine items  & item categories into one dataset for analysis
items = pd.merge(items, item_cats, how='left', on=['item_category_id'])

In [13]:
items.head()

,item_name,item_id,item_category_id,item_category_name_x,item_category_name_y,item_category_name
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40,Кино - DVD,Кино - DVD,Кино - DVD
1,!ABBYY FineReader 12 Professional Edition Full...,1,76,Программы - Для дома и офиса (Цифра),Программы - Для дома и офиса (Цифра),Программы - Для дома и офиса (Цифра)
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40,Кино - DVD,Кино - DVD,Кино - DVD
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40,Кино - DVD,Кино - DVD,Кино - DVD
4,***КОРОБКА (СТЕКЛО) D,4,40,Кино - DVD,Кино - DVD,Кино - DVD


In [15]:
items.shape

(22170, 6)

In [16]:
feature_cnt = 25
TfdfVect = TfidfVectorizer(max_features=feature_cnt)
item_name = pd.DataFrame(TfdfVect.fit_transform(items['item_name']).toarray())

In [18]:
item_name.head()

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.637898,0.0,0.0,...,0.0,0.403761,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.483839
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [ ]:
def custom_merger(dfl, dfr, col_name):
    for column in dfr.columns.values:
        col = col_name + str(column)
        df_left[col] = df_right[column]

In [20]:
model = Sequential()
# define CNN model
model.add(TimeDistributed(Conv2D())
model.add(TimeDistributed(MaxPooling2D()))
model.add(TimeDistributed(Flatten()))
# define LSTM model
model.add(LSTM())
model.add(Dense(1))

SyntaxError: invalid syntax (<ipython-input-20-477a0e2f8241>, line 4)